In [6]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing import image
import os
import random
import shutil
import numpy as np

In [2]:
# 預防錯誤： OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [7]:
# 資料路徑
DATASET_PATH  = 'data/final/kaggle_dogcat'

# 原始訓練集目錄
TRAIN_SET_FOLDER = "train"

# 測試集目錄
TEST_SET_FOLDER = "test"

# 執行訓練集、驗證集目錄
RUNNING_TRAIN_SET_FOLDER = "_train"
RUNNING_VALID_SET_FOLDER = "_val"

# 狗照和貓照目錄
DOG_IMAGE_FOLDER = "dogs"
CAT_IMAGE_FOLDER = "cats"

# 驗證資料佔全部訓練資料的比例
RATIO_OF_VALIDATION_SET = 0.1

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 10

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [ ]:
if os.path.isdir(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER)):
    shutil.rmtree(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER)) 
    
if os.path.isdir(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER)):
    shutil.rmtree(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER)) 

In [ ]:
# 將原始訓練集裡的狗照分成訓練及驗證集
dog_full_path = os.path.join(DATASET_PATH, TRAIN_SET_FOLDER, DOG_IMAGE_FOLDER)
dog_train_path = os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER, DOG_IMAGE_FOLDER)
dog_valid_path = os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER, DOG_IMAGE_FOLDER)

dog_img_files = [f for f in os.listdir(dog_full_path) if os.path.isfile(os.path.join(dog_full_path, f))]
dog_valid_images = random.sample(dog_img_files,  int(len(dog_img_files) * RATIO_OF_VALIDATION_SET))
dog_train_images = [f for f in dog_img_files if f not in dog_valid_images]

os.makedirs(dog_train_path)    
os.makedirs(dog_valid_path)

for img in dog_train_images:
    shutil.copy(os.path.join(dog_full_path, img), dog_train_path)
for img in dog_valid_images:
    shutil.copy(os.path.join(dog_full_path, img), dog_valid_path)    

In [ ]:
# 將原始訓練集裡的貓照分成訓練及驗證集
cat_full_path = os.path.join(DATASET_PATH, TRAIN_SET_FOLDER, CAT_IMAGE_FOLDER)
cat_train_path = os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER, CAT_IMAGE_FOLDER)
cat_valid_path = os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER, CAT_IMAGE_FOLDER)

cat_img_files = [f for f in os.listdir(cat_full_path) if os.path.isfile(os.path.join(cat_full_path, f))]
cat_valid_images = random.sample(cat_img_files,  int(len(cat_img_files) * RATIO_OF_VALIDATION_SET))
cat_train_images = [f for f in cat_img_files if f not in cat_valid_images]

os.makedirs(cat_train_path)    
os.makedirs(cat_valid_path)

for img in cat_train_images:
    shutil.copy(os.path.join(cat_full_path, img), cat_train_path)
for img in cat_valid_images:
    shutil.copy(os.path.join(cat_full_path, img), cat_valid_path)  

In [4]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = image.ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_batches = train_datagen.flow_from_directory(os.path.join(DATASET_PATH, RUNNING_TRAIN_SET_FOLDER),
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = image.ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(os.path.join(DATASET_PATH, RUNNING_VALID_SET_FOLDER),
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Found 3600 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Class #0 = cats
Class #1 = dogs


In [5]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS)

# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

Instructions for updating:
Colocations handled automatically by placer.


/Users/ken/anaconda3/envs/ai100-2/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizationV1) (None, 112, 112, 64) 256         conv1[0][0]                 

bn4f_branch2c (BatchNormalizati (None, 14, 14, 1024) 4096        res4f_branch2c[0][0]             
__________________________________________________________________________________________________
add_12 (Add)                    (None, 14, 14, 1024) 0           bn4f_branch2c[0][0]              
                                                                 activation_36[0][0]              
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 14, 14, 1024) 0           add_12[0][0]                     
__________________________________________________________________________________________________
res5a_branch2a (Conv2D)         (None, 7, 7, 512)    524800      activation_39[0][0]              
__________________________________________________________________________________________________
bn5a_branch2a (BatchNormalizati (None, 7, 7, 512)    2048        res5a_branch2a[0][0]             
__________

Epoch 1/10
450/450 [==============================] - 2444s 5s/step - loss: 0.8055 - acc: 0.7531 - val_loss: 0.1710 - val_acc: 0.9375
Epoch 2/10
450/450 [==============================] - 33550s 75s/step - loss: 0.4237 - acc: 0.8789 - val_loss: 0.1159 - val_acc: 0.9550
Epoch 3/10
450/450 [==============================] - 2411s 5s/step - loss: 0.3440 - acc: 0.9053 - val_loss: 0.1060 - val_acc: 0.9650
Epoch 4/10
450/450 [==============================] - 2600s 6s/step - loss: 0.2255 - acc: 0.9322 - val_loss: 0.0920 - val_acc: 0.9675
Epoch 5/10
450/450 [==============================] - 2376s 5s/step - loss: 0.2235 - acc: 0.9392 - val_loss: 0.1107 - val_acc: 0.9625
Epoch 6/10
450/450 [==============================] - 2371s 5s/step - loss: 0.2630 - acc: 0.9317 - val_loss: 0.1212 - val_acc: 0.9700
Epoch 7/10
450/450 [==============================] - 2560s 6s/step - loss: 0.1859 - acc: 0.9481 - val_loss: 0.0798 - val_acc: 0.9775
Epoch 8/10
450/450 [==============================] - 2391s 

In [31]:
# 從參數讀取圖檔路徑
img_path = os.path.join(DATASET_PATH, TEST_SET_FOLDER)
files = os.listdir(img_path)

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = ['cats', 'dogs']

# 辨識每一張圖
for f in files:
    img = image.load_img(os.path.join(img_path, f), target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    cat_pred = net.predict(x)[0][0]
    print('{},{:.6f}'.format(f[:3], cat_pred))

189,1.000000
162,0.000000
176,0.000126
348,1.000000
360,0.000000
374,1.000000
228,1.000000
214,0.000000
200,0.000115
016,0.000000
002,1.000000
003,0.000000
017,1.000000
201,0.000000
215,0.000000
229,0.000001
375,0.000000
361,1.000000
349,1.000000
177,0.000004
163,0.000000
188,0.000000
149,0.000000
175,0.999968
161,0.000000
388,0.000000
377,1.000000
363,0.008900
203,0.000000
217,1.000000
029,0.000000
001,0.000748
015,0.732554
014,0.998691
000,1.000000
028,0.000000
216,0.000000
202,0.000000
362,1.000000
376,1.000000
389,0.000000
160,0.000000
174,0.000000
148,0.000000
170,0.979358
164,0.000000
158,0.000000
399,1.000000
372,1.000000
366,0.000033
206,1.000000
212,0.000000
004,1.000000
010,0.000000
038,0.000124
039,1.000000
011,0.999935
005,1.000000
213,0.000000
207,0.000126
367,0.000000
373,0.999988
398,1.000000
159,1.000000
165,1.000000
171,1.000000
198,1.000000
167,1.000000
173,0.000000
365,0.999997
371,0.000000
359,0.912916
211,0.000000
205,0.774654
239,0.000000
013,1.000000
007,1.000000

In [30]:
print('160.jpg'[:3])

160
